In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms.algorithms import *
from utils.models import *
from utils.dataloaders import *

from tabulate import tabulate

In [ ]:
device = 'cpu'

# dataset = IncomeDataset(device=device)
dataset = IncomeDataset(device=device, sensitive_feature_labels=["SEX", "AGEP"])

In [5]:
def generate_res():
    test = {'accuracy':[],
            'ei_disparity':[],
            'be_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'be_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'be_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    return train, val, test

def append_res(l,acc,ei,be,dp,eo,eodd):
    l['accuracy'].append(acc)
    l['ei_disparity'].append(ei)
    l['be_disparity'].append(be)
    l['dp_disparity'].append(dp)
    l['eo_disparity'].append(eo)
    l['eodd_disparity'].append(eodd)

def get_res(l):
    res = {}
    res['accuracy_mean'] = np.mean(l['accuracy'])
    res['accuracy_var'] = np.std(l['accuracy'])
    res['accuracy_list'] = l['accuracy']
    res['ei_mean'] = np.mean(l['ei_disparity'])
    res['ei_var'] = np.std(l['ei_disparity'])
    res['ei_list'] = l['ei_disparity']
    res['be_mean'] = np.mean(l['be_disparity'])
    res['be_var'] = np.std(l['be_disparity'])
    res['be_list'] = l['be_disparity']
    res['dp_mean'] = np.mean(l['dp_disparity'])
    res['dp_var'] = np.std(l['dp_disparity'])
    res['dp_list'] = l['dp_disparity']
    res['eo_mean'] = np.mean(l['eo_disparity'])
    res['eo_var'] = np.std(l['eo_disparity'])
    res['eo_list'] = l['eo_disparity']
    res['eodd_mean'] = np.mean(l['eodd_disparity'])
    res['eodd_var'] = np.std(l['eodd_disparity'])
    res['eodd_list'] = l['eodd_disparity']
    return res

def lr_kde_model_runner(dataset, hp, seeds):
    
    train, val, test = generate_res()

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_be_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_be,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_be,results.test_dp,results.test_eo,results.test_eodd)

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fb_model_runner(dataset, hp, seeds):

    train, val, test = generate_res()

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_be_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_be,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_be,results.test_dp,results.test_eo,results.test_eodd)

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fc_model_runner(dataset, hp, seeds):
    
    train, val, test = generate_res()

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fc_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_be_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_be,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_be,results.test_dp,results.test_eo,results.test_eodd)

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, BE_hp_fb, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI_fc = lr_fc_model_runner(dataset, EI_hp_fc, seeds)
    _, _, EI_kde = lr_kde_model_runner(dataset, EI_hp_kde, seeds)
    _, _, BE_fb = lr_fb_model_runner(dataset, BE_hp_fb, seeds)
    _, _, EI_fb = lr_fb_model_runner(dataset, EI_hp_fb, seeds)
    
    return SGD, EI_fc, EI_kde, EI_fb, BE_fb

def fb_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['be_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'be', 'dp', 'eo', 'eodd']))

def kde_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['be_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'be', 'dp', 'eo', 'eodd']))

def fc_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fc_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['be_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'be', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [6]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 0.5

kde_hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [04:47<00:00,  2.87s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [04:47<00:00,  2.88s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [16:08<00:00,  9.68s/epochs]  


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [04:59<00:00,  2.99s/epochs]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei           be         dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  -----------  ---------  ---------  ---------
         0.0001          0          0.808042        0.806497  0.0087619   0.00275443   0.08814    0.0748887  0.0748887
         0.001           0          0.818838        0.815217  0.00690283  0.000700889  0.0719864  0.048219   0.048219
         0.01            0          0.819014        0.815089  0.00518118  0.00179697   0.0752248  0.0584795  0.0584795
         0.1             0          0.811978        0.807647  0.00479581  0.00161182   0.0696276  0.0429922  0.0429922


The learning rate is decided as 0.005.


In [7]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.1
SGD_hp['lambda_'] = 0

#### BE Hyperparameter Test

In [8]:
BE_hp_test = SGD_hp_test.copy()
BE_hp_test['learning_rate'] = [0.1]
BE_hp_test['lambda_'] = [0, 0.1, 0.2, 0.3, 0.4]
BE_hp_test['fairness'] = 'BE'

fb_hyperparameter_test(dataset, BE_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [06:01<00:00,  3.61s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [06:46<00:00,  4.06s/epochs]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [06:52<00:00,  4.13s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [06:59<00:00,  4.20s/epochs]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val           ei           be         dp         eo       eodd
---------------  ---------  ----------------  --------------  -----------  -----------  ---------  ---------  ---------
            0.1        0            0.811978        0.807647  0.00479581   0.00161182   0.0696276  0.0429922  0.0429922
            0.1        0.1          0.808026        0.804293  2.68917e-05  0.000494182  0.0143836  0.0237022  0.0237022
            0.1        0.2          0.807594        0.805379  0.00143615   0.000350649  0.0174078  0.0219771  0.0219771
            0.1        0.3          0.80345         0.799278  0.000687332  0.000132598  0.0149412  0.0286313  0.0286313
            0.1        0.4          0.796375        0.793209  0.000789114  0.00107317   0.0121794  0.0353535  0.0353535


In [9]:
BE_hp_fb = BE_hp_test.copy()
BE_hp_fb['learning_rate'] = 0.1
BE_hp_fb['lambda_'] = 0.2


#### EI Hyperparameter Test

In [10]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EI_hp_test['fairness'] = 'EI'

kde_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [07:04<00:00,  4.25s/epochs]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [06:59<00:00,  4.20s/epochs]


Training finished for all seeds.
training seed 0 started


Training:  84%|████████▍ | 84/100 [05:51<01:11,  4.45s/epochs]

The lambda for EI is decided as 0.9.

In [ ]:
EI_hp_kde = EI_hp_test.copy()
EI_hp_kde['learning_rate'] = 0.1
EI_hp_kde['lambda_'] = 0.95

In [ ]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = [0, 0.6, 0.8, 0.9, 0.95, 0.99]
EI_hp_test['fairness'] = 'EI'

fc_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 19.62epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 19.67epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 19.71epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 19.70epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei          be         dp        eo       eodd
---------------  ---------  ----------------  --------------  ---------  ----------  ---------  --------  ---------
            0.1       0             0.786328        0.792813  0.163427   0.0207947   0.172824   0.228048  0.228048
            0.1       0.6           0.785625        0.790312  0.156853   0.0170713   0.162578   0.213788  0.213788
            0.1       0.8           0.785859        0.79125   0.14002    0.0191092   0.153446   0.203092  0.203092
            0.1       0.9           0.785859        0.791875  0.118302   0.0175867   0.134855   0.17719   0.17719
            0.1       0.95          0.781563        0.790625  0.0768011  0.0238233   0.111644   0.144604  0.144604
            0.1       0.99          0.763281        0.769375  0.0266809  0.00103336  0.0261399  0.041445  0.0996033


In [ ]:
EI_hp_fc = EI_hp_test.copy()
EI_hp_fc['learning_rate'] = 0.1
EI_hp_fc['lambda_'] = 0.99

In [ ]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.1]
EI_hp_test['lambda_'] = [0, 0.1, 0.2, 0.4, 0.6]
EI_hp_test['fairness'] = 'EI'

fb_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 18.89epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 18.89epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 18.98epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:05<00:00, 18.83epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val           ei          be         dp         eo      eodd
---------------  ---------  ----------------  --------------  -----------  ----------  ---------  ---------  --------
            0.1        0            0.786328        0.792813  0.163427     0.0207947   0.172824   0.228048   0.228048
            0.1        0.1          0.75875         0.762813  0.0186442    0.00594016  0.0100629  0.0201127  0.109234
            0.1        0.2          0.755156        0.76125   0.0181578    0.00652522  0.0092852  0.01917    0.10694
            0.1        0.4          0.749062        0.757812  0.00496941   0.0126256   0.0262732  0.0373751  0.130376
            0.1        0.6          0.741563        0.74625   0.000137971  0.0215417   0.0364158  0.0525735  0.128083


In [ ]:
EI_hp_fb = EI_hp_test.copy()
EI_hp_fb['learning_rate'] = 0.1
EI_hp_fb['lambda_'] = 0.1

#### Model training

In [ ]:
seeds = np.arange(1,6)

SGD, EI_fc, EI_kde, EI_fb, BE_fb = experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, BE_hp_fb ,seeds)

/var/folders/mt/mgpv1mld71dfr_kn_fq57str0000gn/T/ipykernel_58492/392662720.py:60: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training: 100%|██████████| 100/100 [00:05<00:00, 18.94epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:04<00:00, 20.31epochs/s]


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:05<00:00, 18.44epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:05<00:00, 18.48epochs/s]

Training finished for all seeds.


In [ ]:
result = []
models = ["SGD", "EI FC", "EI KDE", "EI FB", "BE FB"]
sol = [SGD, EI_fc, EI_kde, EI_fb, BE_fb]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['be_mean'])
    c.append(res['be_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var","be_mean","be_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var     ei_mean      ei_var     be_mean      be_var     dp_mean      dp_var    eo_mean      eo_var    eodd_mean    eodd_var
-------  ---------------  --------------  ----------  ----------  ----------  ----------  ----------  ----------  ---------  ----------  -----------  ----------
SGD              0.7787       0.00161555  0.117516    0.00714839  0.0399493   0.00142264  0.167872    0.00374222  0.23157    0.00478195     0.23157   0.00478195
EI FC            0.74735      0.00261534  0.00327515  0.00192851  0.00444486  0.00279672  0.00543401  0.00384557  0.0269232  0.00693736     0.106832  0.00940244
EI KDE           0.7502       0.00683081  0.00551771  0.003668    0.00775359  0.00622363  0.0136214   0.01421     0.0384817  0.0237088      0.104238  0.0111327
EI FB            0.7544       0.00154596  0.00189271  0.00103583  0.00659287  0.0030006   0.0122688   0.00590121  0.0461318  0.00892781     0.113437  0.0032254
BE FB            0.74755      0.0061